# Worksheet 08

Name: Alexander Miller
UID: U52161825

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [301]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.74306473938678, 3.6051145620135525, 5.08272428384119, 4.739807222866333, 5.444807356893283, 5.88729790361249, 3.697382302169446, 4.833174325197881, 3.2941783744542796, 4.779631252311084]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [302]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.546276256846639, 8.82000871018924, 7.696370457075242, 9.741751095552079, 8.876036333536375, 8.795117393104267, 7.608442641174135, 6.873850172380495, 9.761017484919064, 9.000557907030824]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [303]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.779631252311084, 3.2941783744542796, 9.000557907030824, 9.761017484919064, 4.833174325197881, 3.697382302169446, 5.88729790361249, 6.873850172380495, 7.608442641174135, 5.444807356893283]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

#### The parameters are as such:
Number of componnents: 2
Means: 5, 8
Variances: 1, 1
Number of weights: 10

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [304]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.779631252311084, 3.2941783744542796, 4.833174325197881, 3.697382302169446, 5.88729790361249, 5.444807356893283]
[9.000557907030824, 9.761017484919064, 6.873850172380495, 7.608442641174135]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.656078585773078,  mean_2 = 8.31096705137613
var_1 = 0.8264154991884013,  var_2 = 1.28425681971111


/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The variances aren't great, and the means are alright. They're not bad, but they're not great.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [305]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0] * prob_s[0]) / prob_x )
    prob_s1_x.append( (pdf_i[1] * prob_s[1]) / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.779631252311084
probability of observing that point if it came from cluster 0 =  0.4773732047929539
probability of observing that point if it came from cluster 1 =  0.007086416376553293
point =  3.2941783744542796
probability of observing that point if it came from cluster 0 =  0.12416200735690727
probability of observing that point if it came from cluster 1 =  0.0001508836753097447
point =  9.000557907030824
probability of observing that point if it came from cluster 0 =  4.81486140533924e-07
probability of observing that point if it came from cluster 1 =  0.2689364390477234
point =  9.761017484919064
probability of observing that point if it came from cluster 0 =  2.4993908365995163e-09
probability of observing that point if it came from cluster 1 =  0.16421993649035785
point =  4.833174325197881
probability of observing that point if it came from cluster 0 =  0.4717803089394495
probability of observing that point if it came from cluster 1 =  0.007940301801378622
point =  

The actual probabilites are WAY more accurate than the K-means estimates. The K-means estimates are fine, not great, but the actual probabilities are very close to 1 or 0.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [306]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.653558214216052,  mean_2 = 8.18662990085102
var_1 = 0.8902762055314993,  var_2 = 1.6039891475287462


The means and variances are very similar to the original ones from K means

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [307]:
for i in range(10000):
    # E-step: compute the probability of each sample belonging to each cluster
    prob_s0_x = np.exp(-0.5 * (data - mean[0])**2 / var[0]) / np.sqrt(2 * np.pi * var[0])
    prob_s1_x = np.exp(-0.5 * (data - mean[1])**2 / var[1]) / np.sqrt(2 * np.pi * var[1])
    prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
    prob_s = [prob_s0_x * prob_c[0] / (prob_s0_x * prob_c[0] + prob_s1_x * prob_c[1]), prob_s1_x * prob_c[1] / (prob_s0_x * prob_c[0] + prob_s1_x * prob_c[1])]

    # M-step: update the mean, variance, and prior probability estimates
    mean = [sum(prob_s[0] * data) / sum(prob_s[0]), sum(prob_s[1] * data) / sum(prob_s[1])]
    var = [sum(prob_s[0] * (data - mean[0])**2) / sum(prob_s[0]), sum(prob_s[1] * (data - mean[1])**2) / sum(prob_s[1])]
    prob_s = [sum(prob_s[0]) / len(data), sum(prob_s[1]) / len(data)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.646910012164863,  P(S_2) = 0.35308998783513706
mean_1 = 4.844885839018969,  mean_2 = 8.450618589475154
var_1 = 1.1995243845031074,  var_2 = 1.3305737508239828


The updates converge to the K means estimates.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [308]:
# Compute the hard assignments using the individual probabilities for each data point
cluster_assignments = np.argmax(np.array([prob_s0_x, prob_s1_x]), axis=0)

# Print out the first few assignments for verification
print(cluster_assignments[:10])


[0 0 1 1 0 0 0 1 1 0]
